In [6]:
model(torch.tensor(obs_array, dtype=torch.float))

tensor([[ 0.0287, -0.0252, -0.0534, -0.0450],
        [ 0.0195,  0.1994, -0.0551, -0.3548],
        [ 0.0311, -0.0204, -0.0608, -0.0722],
        ...,
        [-0.0134,  0.4108, -0.1734, -1.2046],
        [ 0.0014,  0.2301, -0.1913, -0.9925],
        [ 0.0226,  0.0544, -0.2043, -0.7890]], grad_fn=<AddmmBackward0>)

In [7]:
obs_array

array([[ 0.01369617, -0.02302133, -0.04590265, -0.04834723],
       [ 0.01323574,  0.17272775, -0.04686959, -0.3551522 ],
       [ 0.0166903 , -0.02169755, -0.05397264, -0.07760915],
       ...,
       [-0.00378081,  0.3945922 , -0.17737961, -1.2545196 ],
       [ 0.00411104,  0.20212865, -0.20247   , -1.0222306 ],
       [ 0.00815361,  0.0101946 , -0.22291462, -0.7993309 ]],
      dtype=float32)

In [ ]:
latents = model.encoder(torch.tensor(obs_list, dtype=torch.float))

# AR

In [5]:
from gran.bprop.model.ar.mdnrnn import MDNRNN

model = MDNRNN().load_from_checkpoint("lightning_logs/vrwd4hg8/checkpoints/epoch=49999-step=50000.ckpt")
data = [obs_array, action_array, rew_array, done_array]
dm = ARDataModule(data=data, batch_size=300)
dm.setup("train")

In [ ]:
dm.dataset[1][0]

In [31]:
model(dm.dataset[0:2][0], torch.tensor([16,8]))[1].shape

torch.Size([24, 5, 4])

In [8]:
a = torch.tensor([[ 0, 18],
        [ 1, 12],
        [ 2,  8],
        [ 3, 19],
        [ 4, 24],
        [ 5,  9],
        [ 6, 45],
        [ 7,  7],
        [ 8, 20],
        [ 9,  8],
        [10, 24],
        [11,  9],
        [12,  8],
        [13, 25],
        [14,  9],
        [15, 20],
        [16, 33],
        [17, 19],
        [18, 11],
        [19,  9]], device='cuda:0')

In [11]:
b = torch.tensor([[ 0, 18]], device='cuda:0')

In [15]:
a[:, 1]

tensor([18, 12,  8, 19, 24,  9, 45,  7, 20,  8, 24,  9,  8, 25,  9, 20, 33, 19,
        11,  9], device='cuda:0')